In [ ]:
import pandas as pd
import numpy as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
import numpy as np
from datetime import datetime

# Read news and behaviors data
col_news = ['NewsId', 'Category', 'SubCat', 'Title', 'Abstract', 'url', 'TitleEnt', 'AbstractEnt']
news_df = pd.read_csv('MINDsmall_train/news.tsv', sep='\t', header=None, names=col_news)

# Read behaviors data
col_behaviors = ['ImpressionID', 'UserID', 'Time', 'History', 'Impressions']
behaviors_df = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None, names=col_behaviors)